In [1]:
import pandas as pd
import glob
import os

In [2]:
path_1 = './baseline_field_mapping'
path_2 = './mapping_tables'
files = glob.glob(os.path.join(path_1, '*.csv')) + glob.glob(os.path.join(path_2, '*.csv'))
files

['./baseline_field_mapping/numeric_prio_fields.csv',
 './baseline_field_mapping/20002_non-cancer_illness.csv',
 './baseline_field_mapping/20004_operation.csv',
 './baseline_field_mapping/20003_treatment_medication.csv',
 './baseline_field_mapping/date_field_lookup.csv',
 './baseline_field_mapping/20001_cancer.csv',
 './baseline_field_mapping/field_id_to_type_concept_id.csv',
 './baseline_field_mapping/discrete_boolean_prio_fields.csv',
 './baseline_field_mapping/discrete_other_fields.csv',
 './baseline_field_mapping/discrete_prio_fields.csv',
 './mapping_tables/gp_prescriptions_drugs_freetext.csv',
 './mapping_tables/hesin_disdest.csv',
 './mapping_tables/gp_clinical_read2_alternative_dot_code_mappings.csv',
 './mapping_tables/gp_clinical_units.csv',
 './mapping_tables/opcs3.csv',
 './mapping_tables/gp_clinical_phenotype_logic.csv',
 './mapping_tables/gender_mapping.csv',
 './mapping_tables/gp_prescriptions_drugs_Read2.csv',
 './mapping_tables/race_mapping.csv',
 './mapping_tables/ctv3

In [3]:
result = []
for file in files:
    table = pd.read_csv(file, comment='#', index_col=False)
    if 'sourceCode' not in table.columns:
        continue
    if 'sourceValueCode' not in table.columns:
        table['sourceValueCode'] = ''
    if sum(table['sourceValueCode'].isna())>1:
        un_codes = table.groupby('sourceCode')['mappingStatus', 'sourceFrequency'].first().reset_index()
    else:
        un_codes = table.groupby(['sourceCode', 'sourceValueCode'])['mappingStatus', 'sourceFrequency'].first().reset_index()
    n_rows = un_codes.shape[0]
    n_appr = len(un_codes[un_codes['mappingStatus'] == 'APPROVED'])
    perc_appr = round(n_appr*100/n_rows, 1)
    n_ign = len(un_codes[un_codes['mappingStatus'] == 'IGNORED'])
    perc_ign = round(n_ign*100/n_rows, 1)
    n_flag = len(un_codes[un_codes['mappingStatus'] == 'FLAGGED'])
    perc_flag = round(n_flag*100/n_rows, 1)
    n_unch = len(un_codes[un_codes['mappingStatus'] == 'UNCHECKED'])
    perc_unch = round(n_unch*100/n_rows, 1)
    freq = un_codes['sourceFrequency'].sum(axis = 0, skipna = True)
    freq_cov = un_codes[(un_codes['mappingStatus'] == 'APPROVED') | (un_codes['mappingStatus'] == 'IGNORED')]['sourceFrequency'].sum(axis = 0, skipna = True)
    perc_freq_cov = round(freq_cov*100/freq, 1)
    result.append([file.split('/')[2].split('.csv')[0], f'{perc_freq_cov}%', n_rows, n_appr, f'{perc_appr}%', n_ign, f'{perc_ign}%',
                   n_flag, f'{perc_flag}%', n_unch, f'{perc_unch}%'])
result

/Library/Python/3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  if __name__ == '__main__':
/Library/Python/3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # This is added back by InteractiveShellApp.init_path()
/Library/Python/3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars


[['numeric_prio_fields',
  '90.6%',
  160,
  138,
  '86.2%',
  1,
  '0.6%',
  21,
  '13.1%',
  0,
  '0.0%'],
 ['20002_non-cancer_illness',
  '86.6%',
  446,
  351,
  '78.7%',
  1,
  '0.2%',
  94,
  '21.1%',
  0,
  '0.0%'],
 ['20004_operation',
  '88.4%',
  254,
  128,
  '50.4%',
  1,
  '0.4%',
  1,
  '0.4%',
  124,
  '48.8%'],
 ['20003_treatment_medication',
  '90.1%',
  6745,
  1330,
  '19.7%',
  1,
  '0.0%',
  74,
  '1.1%',
  5340,
  '79.2%'],
 ['20001_cancer', '72.2%', 82, 48, '58.5%', 1, '1.2%', 33, '40.2%', 0, '0.0%'],
 ['discrete_boolean_prio_fields',
  '96.4%',
  49,
  21,
  '42.9%',
  25,
  '51.0%',
  1,
  '2.0%',
  2,
  '4.1%'],
 ['discrete_other_fields',
  '67.2%',
  123,
  0,
  '0.0%',
  83,
  '67.5%',
  39,
  '31.7%',
  1,
  '0.8%'],
 ['discrete_prio_fields',
  '99.7%',
  1013,
  914,
  '90.2%',
  96,
  '9.5%',
  0,
  '0.0%',
  3,
  '0.3%'],
 ['gp_prescriptions_drugs_freetext',
  '0.0%',
  999,
  0,
  '0.0%',
  0,
  '0.0%',
  0,
  '0.0%',
  999,
  '100.0%'],
 ['hesin_disdes

In [4]:
df_results = pd.DataFrame(result)
df_results.columns = ['file_name', '%_covered', '#_codes', '#_approved', '%_approved', '#_ignored', '%_ignored',
                      '#_flagged', '%_flagged', '#_unchecked', '%_unchecked']
df_results

,file_name,%_covered,#_codes,#_approved,%_approved,#_ignored,%_ignored,#_flagged,%_flagged,#_unchecked,%_unchecked
0,numeric_prio_fields,90.6%,160,138,86.2%,1,0.6%,21,13.1%,0,0.0%
1,20002_non-cancer_illness,86.6%,446,351,78.7%,1,0.2%,94,21.1%,0,0.0%
2,20004_operation,88.4%,254,128,50.4%,1,0.4%,1,0.4%,124,48.8%
3,20003_treatment_medication,90.1%,6745,1330,19.7%,1,0.0%,74,1.1%,5340,79.2%
4,20001_cancer,72.2%,82,48,58.5%,1,1.2%,33,40.2%,0,0.0%
5,discrete_boolean_prio_fields,96.4%,49,21,42.9%,25,51.0%,1,2.0%,2,4.1%
6,discrete_other_fields,67.2%,123,0,0.0%,83,67.5%,39,31.7%,1,0.8%
7,discrete_prio_fields,99.7%,1013,914,90.2%,96,9.5%,0,0.0%,3,0.3%
8,gp_prescriptions_drugs_freetext,0.0%,999,0,0.0%,0,0.0%,0,0.0%,999,100.0%
9,hesin_disdest,62.8%,78,49,62.8%,0,0.0%,0,0.0%,29,37.2%


In [5]:
df_results.to_csv('mapping_metrics.csv', sep=',', index=False)